# Fine-tune Llama 3.1 8B with QLoRA and SageMaker remote decorator for Automotive domain

---

Fine-tune the Meta-Llama 3.1 8B model using QLoRA, Hugging Face PEFT, and bitsandbytes for the Automotive Domain.

Dataset:
* [sp01/Automotive_NER](https://huggingface.co/datasets/sp01/Automotive_NER)

We are using SageMaker remote decorator for runinng the fine-tuning job on Amazon SageMaker Training job
---

JupyterLab Instance Type: ml.t3.medium

Fine-Tuning:
* Instance Type: ml.g5.48xlarge

Install the required libriaries, including the Hugging Face libraries, and restart the kernel.

In [ ]:
import os, sys
os.environ["PATH"] = f"{os.path.dirname(sys.executable)}:" + os.environ["PATH"]
!which python
p=!python -V
p = p[0].split(" ")[1].split(".")
if p[0] != "3" or p[1] != "11":
    raise EnvironmentError("This notebook requires Python 3.11")
else:
    print("all good")

/Library/Developer/CommandLineTools/usr/bin/python3


In [ ]:
# test to see if we are in the right python environment
!python -V

Python 3.9.6


In [ ]:
%pip install -r requirements.txt

In [ ]:
%pip install -q -U boto3
%pip install -q -U botocore
%pip install -q -U datasets==2.20.0
%pip install -q -U scikit-learn==1.5.1
%pip install -q -U sagemaker==2.232.2 
%pip install -q -U cloudpickle==2.2.1
%pip uninstall -y -q torch torchvision torchaudio
%pip install -q -U --index-url https://download.pytorch.org/whl/cu124 \
    torch==2.4.0+cu124 torchvision==0.19.0+cu124 torchaudio==2.4.0+cu124

# 2) Keep the serializer and SDK aligned with the container
%pip install -q -U cloudpickle==2.2.1 sagemaker==2.232.2 accelerate==0.34.2 transformers==4.45.1


## Setup Configuration file path

We are setting the directory in which the config.yaml file resides so that remote decorator can make use of the settings through [SageMaker Defaults](https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk).

This notebook is using the Hugging Face container for the `us-east-1` region. Make sure you are using the right image for your AWS region, otherwise edit [config.yaml](./config.yaml). Container Images are available [here](https://github.com/aws/deep-learning-containers/blob/master/available_images.md)


In [ ]:
import os

# Set path to config file
os.environ["SAGEMAKER_USER_CONFIG_OVERRIDE"] = os.getcwd()

## Visualize the dataset

Read train dataset from Hugging Face datasets and convert DatasetDict to Pandas Dataframe or read the sample dataset from the generated csv file

In [ ]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("sp01/Automotive_NER")
df = pd.DataFrame(dataset['train'])

df.head()

## Data analysis and distribution

The following cells should be run if you want to perform a complete analysis of the dataset

### Analyze word frequency 

We can use the TfidfVectorizer from the scikit-learn library to tokenize the text and select the top and bottom words in the dataset based on how important a word is to a document in the corpus.

We are going to run an Amazon SageMaker job for returning the computed `top_words` and `bottom_words`

In [ ]:
import numpy as np
import re
from sagemaker.remote_function import remote
from sklearn.feature_extraction.text import TfidfVectorizer
import string

@remote(volume_size=10, job_name_prefix=f"preprocess-auto-ner-auto-merge", instance_type="ml.m4.10xlarge")
def preprocess(df, 
               top_n=6000,
               bottom_n=6000
    ):
    # Download nltk stopwords
    import nltk
    nltk.download('stopwords')
    from nltk.corpus import stopwords

    # Define a function to preprocess text
    def preprocess_text(text):
        if not isinstance(text, str):
            # Return an empty string or handle the non-string value as needed
            return ''
    
        # Remove punctuation
        text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    
        # Convert to lowercase
        text = text.lower()
    
        # Remove stop words (optional)
        stop_words = set(stopwords.words('english'))
        text = ' '.join([word for word in text.split() if word not in stop_words])
    
        return text
    
    print("Applying text preprocessing")
    
    # Preprocess the text columns
    df['DESC_DEFECT'] = df['DESC_DEFECT'].apply(preprocess_text)
    df['CONEQUENCE_DEFECT'] = df['CONEQUENCE_DEFECT'].apply(preprocess_text)
    df['CORRECTIVE_ACTION'] = df['CORRECTIVE_ACTION'].apply(preprocess_text)
    
    # Create a TfidfVectorizer object
    tfidf_vectorizer = TfidfVectorizer()

    print("Compute TF-IDF")
    
    # Fit and transform the text data
    X_tfidf = tfidf_vectorizer.fit_transform(df['DESC_DEFECT'] + ' ' + df['CONEQUENCE_DEFECT'] + ' ' + df['CORRECTIVE_ACTION'])
    
    # Get the feature names (words)
    feature_names = tfidf_vectorizer.get_feature_names_out()
    
    # Get the TF-IDF scores
    tfidf_scores = X_tfidf.toarray()
    
    top_word_indices = np.argsort(tfidf_scores.sum(axis=0))[-top_n:]
    bottom_word_indices = np.argsort(tfidf_scores.sum(axis=0))[:bottom_n]

    print("Extracting top and bottom words")
    
    # Get the top and bottom words
    top_words = [feature_names[i] for i in top_word_indices]
    bottom_words = [feature_names[i] for i in bottom_word_indices]

    return top_words, bottom_words

In [ ]:
top_words, bottom_words = preprocess(df)

### Analyze the distributions per importance 

We can now analyze the distributions by sorting the word frequencies and examining the most common words and phrases.

In [ ]:
# Print the top 50 elements from top_words and bottom_words
print("Top 10 words:")
for word in top_words[:10]:
    print(word)

| Top 10 words|
|-------------|
| azure |
| rondo |
| f65277 |
| stressed |
| ocr |
| middle |
| 18017 |
| 7060 |
| interaxle |
| seep |

In [ ]:
print("Top 10 words in bottom:")
for word in bottom_words[:10]:
    print(word)

| Top 10 words in bottom: |
| ------------------------|
| 22x01431gm |
| 22x01761gm |
| 22x00981gm |
| 22x01421gm |
| purchsed |
| 22x01861gm |
| 18602294884 |
| o22x00591gm |
| 22x01821gm |
| 22x01781gm |

Utility function for identify the rows that contains rare or important words

In [ ]:
# Create a function to check if a row contains important or rare words
def contains_important_or_rare_words(row):
    try:
        if ("DESC_DEFECT" in row.keys() and row["DESC_DEFECT"] is not None and
            "CONEQUENCE_DEFECT" in row.keys() and row["CONEQUENCE_DEFECT"] is not None and
            "CORRECTIVE_ACTION" in row.keys() and row["CORRECTIVE_ACTION"] is not None):
            text = row['DESC_DEFECT'] + ' ' + row['CONEQUENCE_DEFECT'] + ' ' + row['CORRECTIVE_ACTION']
        
            text_words = set(text.split())
        
            # Check if the row contains any important words (top_words)
            for word in top_words:
                if word in text_words:
                    return 'top'
        
            # Check if the row contains any rare words (bottom_words)
            for word in bottom_words:
                if word in text_words:
                    return 'bottom'
        
            return 'neither'
        else:
            return 'none'
    except Exception as e:
        raise e

Filter the dataset to keep only rows with most rare and important words

In [ ]:
# Apply the function to the DataFrame and create a new column 'word_type'
df['word_type'] = df.apply(contains_important_or_rare_words, axis=1)

# Select all rows from each group
top_rows = df[df['word_type'] == 'top']
bottom_rows = df[df['word_type'] == 'bottom']

# Combine the two groups, ensuring a balanced dataset
if len(bottom_rows) > 0:
    df = pd.concat([top_rows, bottom_rows.sample(n=len(bottom_rows), random_state=42)], ignore_index=True)
else:
    df = top_rows.copy()

# If the combined dataset has fewer than 6010 rows, fill with remaining rows
if len(df) < 6010:
    remaining_rows = df[df['word_type'] == 'neither'].sample(n=6010 - len(df), random_state=42)
    df = pd.concat([df, remaining_rows], ignore_index=True)

df = df.sample(n=6010, random_state=42)

df.to_csv('sample_dataset.csv', index=False)

***

## Generate train, and test datasets

If the data processing step was executed at least one time, we will find the dataset `sample_dataset.csv` ready to be used. Execute again the data processing step in case of changes in the processing script

In [ ]:
import pandas as pd

df = pd.read_csv("./sample_dataset.csv")

Split the dataset in train and test

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.1, random_state=42)

train, valid = train_test_split(train, test_size=10, random_state=42)

print("Number of train elements: ", len(train))
print("Number of test elements: ", len(test))
print("Number of validation elements: ", len(valid))

## Prepare prompt templates for fine-tuning

Create a prompt template for consequences of a defect

In [ ]:
from random import randint

# template dataset to add prompt to each sample
def template_dataset_consequence(sample):
    # custom instruct prompt start
    prompt_template = f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>These are the information related to the defect:\nManufacturer: {{mfg_name}}\nComponent: {{comp_name}}\nDescription of the defect: {{desc_defect}}\n\n\nWhat are the consequences of defect?<|eot_id|><|start_header_id|>assistant<|end_header_id|>{{consequence_defect}}<|end_of_text|><|eot_id|>"""
    sample["text"] = prompt_template.format(
        mfg_name=sample["MFGNAME"],
        comp_name=sample["COMPNAME"],
        desc_defect=sample["DESC_DEFECT"].lower(),
        consequence_defect=sample["CONEQUENCE_DEFECT"].lower())
    return sample

Create a prompt template for corrective action

In [ ]:
from random import randint

# template dataset to add prompt to each sample
def template_dataset_corrective_action(sample):
    # custom instruct prompt start
    prompt_template = f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>These are the information related to the defect:\nManufacturer: {{mfg_name}}\nComponent: {{comp_name}}\nDescription of the defect: {{desc_defect}}\n\n\nWhat are the possible corrective actions?<|eot_id|><|start_header_id|>assistant<|end_header_id|>{{corrective_action}}<|end_of_text|><|eot_id|>"""
    sample["text"] = prompt_template.format(
        mfg_name=sample["MFGNAME"],
        comp_name=sample["COMPNAME"],
        desc_defect=sample["DESC_DEFECT"].lower(),
        corrective_action=sample["CORRECTIVE_ACTION"].lower())
    return sample

Generate the train and test datasets containing the consequences of a defect

In [ ]:
from datasets import Dataset, DatasetDict

train_dataset = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)

dataset = DatasetDict({"train": train_dataset, "test": test_dataset})

train_dataset_consequence = dataset["train"].map(template_dataset_consequence, remove_columns=list(dataset["train"].features))

print(train_dataset_consequence[randint(0, len(dataset))]["text"])

test_dataset_consequence = dataset["test"].map(template_dataset_consequence, remove_columns=list(dataset["test"].features))

Generate the train and test datasets containing the possible corrective actions of a defect

In [ ]:
from datasets import Dataset, DatasetDict

train_dataset = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)

dataset = DatasetDict({"train": train_dataset, "test": test_dataset})

train_dataset_actions = dataset["train"].map(template_dataset_corrective_action, remove_columns=list(dataset["train"].features))

print(train_dataset_actions[randint(0, len(dataset))]["text"])

test_dataset_actions = dataset["test"].map(template_dataset_corrective_action, remove_columns=list(dataset["test"].features))

Now let's concatenate and generate the final train and test datasets

In [ ]:
from datasets import concatenate_datasets

train_dataset = concatenate_datasets([train_dataset_consequence, train_dataset_actions])
test_dataset = concatenate_datasets([test_dataset_consequence, test_dataset_actions])

print("Number of train elements: ", len(train_dataset))
print("Number of test elements: ", len(test_dataset))



To train our model, we need to convert our inputs (text) to token IDs. This is done by a Hugging Face Transformers Tokenizer. In addition to QLoRA, we will use bitsanbytes 4-bit precision to quantize out frozen LLM to 4-bit and attach LoRA adapters on it.



In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

Define the train function

In [ ]:
# TODO: Replace with your Hugging Face token
%set_env HF_TOKEN=<your_huggingface_token_here>

In [ ]:
# FSDP configuration for Llama-3.1-8B layers
fsdp = "full_shard auto_wrap"
fsdp_config = {
    "fsdp_transformer_layer_cls_to_wrap": "LlamaDecoderLayer"
}

In [ ]:
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

Provide the ARN of the tracking server that you want to track your training job with

In [ ]:
from accelerate import Accelerator
from huggingface_hub import login
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model
from sagemaker.remote_function import remote
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed
import transformers

# Core training implementation (no decorator)
def train_impl(
        model_name,
        train_ds,
        test_ds=None,
        lora_r=8,
        lora_alpha=16,
        lora_dropout=0.1,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        gradient_accumulation_steps=1,
        learning_rate=2e-4,
        num_train_epochs=1,
        fsdp="",
        fsdp_config=None,
        gradient_checkpointing=False,
        merge_weights=False,
        seed=42,
        token=None
):

    set_seed(seed)

    accelerator = Accelerator()

    if token is not None:
        login(token=token)

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    # Enforce 2048 token context with truncation
    tokenizer.model_max_length = 2048

    # Set Tokenizer pad Token
    tokenizer.pad_token = tokenizer.eos_token

    with accelerator.main_process_first():
        # tokenize and chunk dataset
        lm_train_dataset = train_ds.map(
            lambda sample: tokenizer(sample["text"], max_length=2048, truncation=True), remove_columns=list(train_ds.features)
        )

        print(f"Total number of train samples: {len(lm_train_dataset)}")

        if test_ds is not None:
            lm_test_dataset = test_ds.map(
                lambda sample: tokenizer(sample["text"], max_length=2048, truncation=True), remove_columns=list(test_ds.features)
            )

            print(f"Total number of test samples: {len(lm_test_dataset)}")
        else:
            lm_test_dataset = None

    torch_dtype = torch.bfloat16

    # Defining additional configs for FSDP
    if fsdp != "" and fsdp_config is not None:
        model_configs = {
            "torch_dtype": torch_dtype
        }

        fsdp_configurations = {
            "fsdp": fsdp,
            "fsdp_config": fsdp_config,
            "gradient_checkpointing_kwargs": {
                "use_reentrant": False
            },
            "tf32": True
        }
    else:
        model_configs = {"torch_dtype": torch_dtype}
        fsdp_configurations = dict()

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        attn_implementation="flash_attention_2",
        use_cache=not gradient_checkpointing,
        cache_dir="/tmp/.cache",
        **model_configs
    )

    if gradient_checkpointing:
        model.gradient_checkpointing_enable()

    config = LoraConfig(
        r=lora_r,
        lora_alpha=lora_alpha,
        target_modules="all-linear",
        lora_dropout=lora_dropout,
        bias="none",
        task_type="CAUSAL_LM"
    )

    model = get_peft_model(model, config)
    print_trainable_parameters(model)

    trainer = transformers.Trainer(
        model=model,
        train_dataset=lm_train_dataset,
        eval_dataset=lm_test_dataset if lm_test_dataset is not None else None,
        args=transformers.TrainingArguments(
            per_device_train_batch_size=per_device_train_batch_size,
            per_device_eval_batch_size=per_device_eval_batch_size,
            gradient_accumulation_steps=gradient_accumulation_steps,
            gradient_checkpointing=gradient_checkpointing,
            logging_strategy="steps",
            logging_steps=1,
            log_on_each_node=False,
            num_train_epochs=num_train_epochs,
            learning_rate=learning_rate,
            bf16=True,
            ddp_find_unused_parameters=False,
            save_strategy="no",
            output_dir="outputs",
            **fsdp_configurations
        ),
        data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
    )

    trainer.train()

    if trainer.is_fsdp_enabled:
        trainer.accelerator.state.fsdp_plugin.set_state_dict_type("FULL_STATE_DICT")

    if merge_weights:
        output_dir = "/tmp/model"

        # save model with adapters (bf16 full precision)
        trainer.model.save_pretrained(output_dir, safe_serialization=False)

        if accelerator.is_main_process:
            # clear memory
            del model
            del trainer

            torch.cuda.empty_cache()

            # load PEFT model
            model = AutoPeftModelForCausalLM.from_pretrained(
                output_dir,
                torch_dtype=torch.float16,
                low_cpu_mem_usage=True,
                trust_remote_code=True,
            )

            # Merge LoRA and base model and save
            model = model.merge_and_unload()
            model.save_pretrained(
                "/opt/ml/model", safe_serialization=True, max_shard_size="2GB"
            )
    else:
        trainer.model.save_pretrained("/opt/ml/model", safe_serialization=True)

    if accelerator.is_main_process:
        tokenizer.save_pretrained("/opt/ml/model")

# Factory to create a RemoteFunction-wrapped trainer with a unique job name

def make_train_fn(job_name_prefix: str, instance_type: str = "ml.g5.48xlarge"):
    return remote(
        keep_alive_period_in_seconds=0,
        volume_size=100,
        job_name_prefix=job_name_prefix,
        use_torchrun=True,
        nproc_per_node=8,
        instance_type=instance_type,
    )(train_impl)

In [ ]:
import sys, torch, cloudpickle, sagemaker, accelerate, transformers
print(sys.version)            # 3.11.x
print(torch.__version__)      # 2.4.0+cu124
print(cloudpickle.__version__)# 2.2.1
print(sagemaker.__version__)  # 2.232.2
print(accelerate.__version__) # 0.34.2
print(transformers.__version__) # 4.45.1

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed

# Define a small grid of LoRA configs
lora_grid = [
    {"name":"r8a16d05",  "lora_r":8,  "lora_alpha":16, "lora_dropout":0.05},
    {"name":"r16a32d10", "lora_r":16, "lora_alpha":32, "lora_dropout":0.10},
]

# Helper to launch one job with a unique prefix

def launch_one(cfg):
    prefix = f"train-{cfg['name']}"
    fn = make_train_fn(job_name_prefix=prefix, instance_type="ml.g5.48xlarge")
    return fn(
        model_id,
        train_ds=train_dataset,
        test_ds=test_dataset,
        lora_r=cfg["lora_r"],
        lora_alpha=cfg["lora_alpha"],
        lora_dropout=cfg["lora_dropout"],
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        gradient_accumulation_steps=1,
        gradient_checkpointing=True,
        num_train_epochs=1,
        fsdp=fsdp,
        fsdp_config=fsdp_config,
        merge_weights=True,
        token=os.environ.get("HF_TOKEN")
    )

# Fire runs in parallel (each uses its own ml.g5.48xlarge)
with ThreadPoolExecutor(max_workers=len(lora_grid)) as ex:
    futs = {ex.submit(launch_one, cfg): cfg for cfg in lora_grid}
    for fut in as_completed(futs):
        cfg = futs[fut]
        try:
            fut.result()
            print(f"Completed: train-{cfg['name']}")
        except Exception as e:
            print(f"Failed train-{cfg['name']}: {e}")

